# What Affects the Ratings?

**By Li-Yen Hsu (11/08/2017)**
<br><br>
  In another [kernel](https://www.kaggle.com/liyenhsu/simple-content-based-recommenders), I implemented two content-based recommender systems and tried the [SVD algorithm](http://sifter.org/simon/journal/20061211.html) using [Surprise](http://surpriselib.com/). In this kernel, I will focus on data exploration, looking for correlations within the data.
<br><br>
  It's obvious that the main factors that affect the ratings are (1) how delicious the food is, and (2) how well a customer is served. These two are definitely not something that are in the data. But some of the features of a restaurant still have some influence on the ratings. For a content-based approach, selecting the relevant features is important for the rating prediction.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random

pd.set_option('mode.chained_assignment', None)

# Note that there are no NANs in these data; '?' is
# used when there is missing information
accepts = pd.read_csv('../input/chefmozaccepts.csv')
cuisine = pd.read_csv('../input/chefmozcuisine.csv')
hours = pd.read_csv('../input/chefmozhours4.csv')
parking = pd.read_csv('../input/chefmozparking.csv')
geo = pd.read_csv('../input/geoplaces2.csv') 
usercuisine = pd.read_csv('../input/usercuisine.csv')
payment = pd.read_csv('../input/userpayment.csv')
profile = pd.read_csv('../input/userprofile.csv')
rating = pd.read_csv('../input/rating_final.csv')

## Data Preprocessing ##
This part is pretty much the same as another [kernel](https://www.kaggle.com/liyenhsu/simple-content-based-recommenders) I posted. Please go there for a more complete description.

In [ ]:
res_all = np.concatenate((accepts.placeID, cuisine.placeID, 
                          hours.placeID, parking.placeID, geo.placeID))
res_all = np.sort( np.unique(res_all) ) # All the placeID's

print("There are {} restaurants.".format(len(res_all)))

In [ ]:
user_all = np.concatenate((usercuisine.userID, payment.userID, profile.userID))
user_all = np.sort( np.unique(user_all) ) # All the userID's

print("There are {} users.".format(len(user_all)))

In [ ]:
rating.head(10) # There are three types of ratings

In [ ]:
print("There are {} restaurants with ratings.".format(len(rating.placeID.unique())))
print("There are {} users who gave ratings.".format(len(rating.userID.unique())))

I will now create three data frames for the three types of rating, each of which will have a shape of (938, 138). A restaurant-user pair without a rating will be recorded as -1 (to be different from 0, the lowest rating).

In [ ]:
overall_rating = pd.DataFrame( np.zeros((len(res_all),len(user_all)))-1.0, 
                              columns=user_all, index=res_all )
food_rating = overall_rating.copy()
service_rating = overall_rating.copy() 

for r, u, o, f, s in zip(rating.placeID, rating.userID, rating.rating, rating.food_rating, 
                         rating.service_rating):
    overall_rating.loc[r,u] = o
    food_rating.loc[r,u] = f
    service_rating.loc[r,u] = s

# This tells us whether a restaurant-user pair has a rating. 0 means No and 1 means Yes.
review = pd.DataFrame( np.zeros(overall_rating.shape), columns=user_all, index=res_all)
review[overall_rating >= 0] = 1

I'll be looking at the restaurant features from **cuisine, parking** and **geo**. The payment options and business hours should have very little effects on the rating, so I won't look into **accepts** and **hours** here.

In [ ]:
cuisine.head(10)

In [ ]:
# use dummy variables for different cuisine categories of the restaurants
res_cuisine = pd.get_dummies(cuisine,columns=['Rcuisine'])

# remove duplicate restaurant ID's. 
# A restaurant with multiple cuisine categories would have multiple columns equal 1
res_cuisine = res_cuisine.groupby('placeID',as_index=False).sum()

In [ ]:
parking.head(10)

In [ ]:
# use dummy variables for different cuisine categories of the restaurants
res_parking = pd.get_dummies(parking,columns=['parking_lot'])

# remove duplicate restaurant ID's. 
# A restaurant with multiple parking options would have multiple columns equal 1
res_parking = res_parking.groupby('placeID',as_index=False).sum()

Let's select some columns from **geo** that might affect the ratings.

In [ ]:
geo.columns.values

In [ ]:
geo.head()

In [ ]:
# These are the ones that I think might be relevant
res_features = geo[['placeID','alcohol','smoking_area','other_services','price','dress_code',
                         'accessibility','area']]

df_res = pd.DataFrame({'placeID': res_all})
df_res = pd.merge(left=df_res, right=res_cuisine, how="left", on="placeID")
df_res = pd.merge(left=df_res, right=res_parking, how="left", on="placeID")
df_res = pd.merge(left=df_res, right=res_features, how="left", on="placeID")

From now on I will use four (130,138) arrays, **R, Y_overall, Y_food** and **Y_service**, for their ratings, where the axis=0 dimension is for the restaurants and the axis=1 dimension is for the consumers. The elements of **R** are either 0 or 1, showing whether a restaurant-user pair has a rating. The information of the restaurants will be recorded in a (130,n_feature) array, **X**.

In [ ]:
# The placeID's for the 130 restaurants with ratings
res_rated = res_all[np.sum(review,axis=1) > 0] 

# tells us whether a restaurant-user pair has a rating. 0 means No and 1 means Yes.
R = review.loc[res_rated].values  # shape = (130,138)

# Now these have a shape of (130, 138) too
Y_overall = overall_rating.loc[res_rated].values
Y_food  = food_rating.loc[res_rated].values
Y_service = service_rating.loc[res_rated].values

# select the indices of "df_res" where a restaurant has ratings
index = [x in res_rated for x in df_res['placeID'].values] #np.array()

# restaurant features for the 130 restaurants with ratings
X = df_res.loc[index, :].reset_index(drop=True)

X.isnull().sum() # all the NANs are from cuisine 

In [ ]:
# fill all NANs with 0
X = X.fillna(0) 

# drop a feature if the entire column are 0
features_to_drop = X.columns.values[np.sum(X,axis=0) == 0] 
X = X.drop(features_to_drop, axis=1)

# drop placeID
X = X.drop(['placeID'], axis=1)

# There are the restaurant features we'll explore
X.columns.values 

Now let's select some of the user features that are related to the restaurant features

In [ ]:
profile.columns.values

So **smoker**, **drink_level**, **transport**, **budget** and **dress_preference** are related to **smoking_area**, **alcohol**, **parking options** (**parking_lot_none**, **parking_lot_public**, **parking_lot_valet parking** and **parking_lot_yes**), **price**, and **dress_code**

In [ ]:
user_info = profile[['smoker','drink_level','transport','budget','dress_preference']] 

print(user_info.smoker.value_counts())
print('\n')
print(user_info.drink_level.value_counts())
print('\n')
print(user_info.transport.value_counts())
print('\n')
print(user_info.budget.value_counts())
print('\n')
print(user_info.dress_preference.value_counts())

Before exploring these data, I will combine 'public' and 'on foot' of **transport** into just one category, 'no car', because these two types of people would feel the same about whether there is a parking lot. Similarly, 'elegant' and 'formal' of **dress_preference** will be merged to just one category, 'formal'.

In [ ]:
user_info.transport = user_info.transport.replace({'public':'no car', 'on foot':'no car'})
user_info.dress_preference = user_info.dress_preference.replace({'elegant':'formal'})

## Exploratory Visualization
In order to see how the restaurant features affect the ratings, I will first get the mean rating for each restaurant. And then for each feature, I will show how the rating changes with different feature values. To do this, I will group the restaurants based on the different feature values and then average the mean ratings mentioned above for each group. Notice that what I do is **NOT** averaging all the ratings with the feature equals to a certain value. Instead, it is first getting the mean rating for each restaurant and then grouping. If I used the former approach, the mean rating of a group would be biased towards the restaurants with more ratings. This should be avoided because we want to treat every restaurant equally.

In [ ]:
# Calculate the mean rating for each restaurant
def GetMean(Y,R):

    Y = Y*R
    mean =  (np.sum(Y, axis=1)/np.sum((R == 1.0), axis=1)).reshape(Y.shape[0],1)
    return mean

Y_overall_mean = GetMean(Y_overall,R)
Y_food_mean = GetMean(Y_food,R)
Y_service_mean = GetMean(Y_service,R)

In [ ]:
# This is the function I'll use to plot and print the mean ratings of different 
# groups of restaurants based on different values of a given feature
def plot_mean_rating(df,rotate=False):
    
    n = df.shape[1]
    columns = df.columns.values
    
    if n > 1:
        y_overall = [ Y_overall_mean[df[i] == 1].mean() for i in columns ]
        y_food = [ Y_food_mean[df[i] == 1].mean() for i in columns ]
        y_service = [ Y_service_mean[df[i] == 1].mean() for i in columns ] 
        y = pd.DataFrame({'overall':y_overall, 'food':y_food, 'service':y_service},
                         columns=['overall','food','service'],index=columns) 
        ticks = columns
        
    else:
        values = df[columns[0]].unique()
        values = values[values != '?']
        y_overall = [ Y_overall_mean[df[columns[0]] == i].mean() for i in values ]
        y_food = [ Y_food_mean[df[columns[0]] == i].mean() for i in values ]
        y_service = [ Y_service_mean[df[columns[0]] == i].mean() for i in values ] 
        y = pd.DataFrame({'overall':y_overall, 'food':y_food, 'service':y_service},
                         columns=['overall','food','service'],index=values) 
        ticks = values
   
    fig = plt.figure()
    plt.plot(range(y.shape[0]),y['overall'],'-o',c='k',label='overall')
    plt.plot(range(y.shape[0]),y['food'],'-o',c='r',label='food')   
    plt.plot(range(y.shape[0]),y['service'],'-o',c='b',label='service')
    plt.xticks(range(y.shape[0]),ticks,fontsize=13)
    if rotate: plt.xticks(rotation=40)
    plt.yticks(fontsize=13) 
    if n == 1: plt.xlabel(columns[0],fontsize=15)
    plt.ylabel('mean rating',fontsize=15)
    plt.legend(fontsize=15,frameon=False)
    plt.show()
    
    print(y)

In [ ]:
# This function is similar to the one above, but this time the result is split
# into different groups of users as well
def plot_mean_rating_split(df,userinfo,rotate=False):
    
    n = df.shape[1]
    columns = df.columns.values
    
    cases = userinfo.unique()
    cases = cases[cases != '?']
    
    num = len(cases)
    y = {}
    
    if n > 1:
        
        for i in range(num):
            
            index = (userinfo == cases[i])
            R_case = np.zeros(R.shape)
            R_case[:,index] = R[:,index]

            Y_overall_case = GetMean(Y_overall,R_case)
            Y_food_case = GetMean(Y_food,R_case)
            Y_service_case = GetMean(Y_service,R_case)
        
            isnan = np.isnan(Y_overall_case).reshape(-1)
            y_overall = [Y_overall_case[(df[j] == 1) & 
                        (isnan == False)].mean() for j in columns]
            y_food = [Y_food_case[(df[j] == 1) & 
                     (isnan == False)].mean() for j in columns]
            y_service = [Y_service_case[(df[j] == 1) & 
                        (isnan == False)].mean() for j in columns]
            
            y[cases[i]] = pd.DataFrame({'overall':y_overall, 'food':y_food, 'service':
                          y_service}, columns=['overall','food','service'],index=columns)
            
        ticks = columns

     
    else:
        
        for i in range(num):
            
            values = df[columns[0]].unique()
            values = values[values != '?']
            
            index = (userinfo == cases[i])
            R_case = np.zeros(R.shape)
            R_case[:,index] = R[:,index]

            Y_overall_case = GetMean(Y_overall,R_case)
            Y_food_case = GetMean(Y_food,R_case)
            Y_service_case = GetMean(Y_service,R_case)
        
            isnan = np.isnan(Y_overall_case).reshape(-1)
            y_overall = [Y_overall_case[(df[columns[0]] == j) & 
                        (isnan == False)].mean() for j in values]
            y_food = [Y_food_case[(df[columns[0]] == j) & 
                     (isnan == False)].mean() for j in values]
            y_service = [Y_service_case[(df[columns[0]] == j) & 
                        (isnan == False)].mean() for j in values]
                        
            y[cases[i]] = pd.DataFrame({'overall':y_overall, 'food':y_food, 'service':
                          y_service}, columns=['overall','food','service'],index=values)

        ticks = values
   

    f, (ax1, ax2, ax3) = plt.subplots(1,3, sharex=True, sharey=True, figsize=(24,6))
 
    color = ['k','r','b'] 

    for i in range(num):
        ax1.plot(range(len(ticks)),y[cases[i]]['overall'],'-o',c=color[i],label=cases[i])
        ax2.plot(range(len(ticks)),y[cases[i]]['food'],'-o',c=color[i],label=cases[i])
        ax3.plot(range(len(ticks)),y[cases[i]]['service'],'-o',c=color[i],label=cases[i])
    
    ax1.set_title('overall',fontsize=20)
    ax2.set_title('food',fontsize=20)
    ax3.set_title('service',fontsize=20)

    ax1.tick_params(labelsize=16)
    ax2.tick_params(labelsize=16)
    ax3.tick_params(labelsize=16)
    
    if rotate:
        ax1.set_xticks(range(len(ticks)))
        ax1.set_xticklabels(ticks, rotation=40)
        ax2.set_xticks(range(len(ticks)))
        ax2.set_xticklabels(ticks, rotation=40)
        ax3.set_xticks(range(len(ticks)))
        ax3.set_xticklabels(ticks, rotation=40)
    else:
        plt.xticks(range(len(ticks)),ticks)
                           
    if n == 1: 
        ax2.set_xlabel(columns[0],fontsize=20)
    
    ax1.set_ylabel('mean rating',fontsize=20)
    
    plt.legend(fontsize=20,frameon=False)
    plt.show()
    
    return y

Now I'll go over all the restaurant features that are stored in **X**

### Price

In [ ]:
X.price.value_counts()

In [ ]:
user_info.budget.value_counts()

In [ ]:
# mean rating as a function of price
plot_mean_rating(X[['price']])

In [ ]:
# mean rating as a function of price, split into three groups of consumers
# with low, medium, and high budget
y = plot_mean_rating_split(X[['price']],user_info.budget)

print('low budget:')
print(y['low'])
print('\nmedium budget:')
print(y['medium'])
print('\nhigh budget:')
print(y['high'])

**Observations: ** You get what you pay for? In general, restaurants with medium or high price have higher ratings (high ~ medium > low). However, the consumers with high budget actually prefer medium-price restaurants over high-price ones.

### Parking Options

In [ ]:
columns = ['parking_lot_none','parking_lot_public', 'parking_lot_valet parking','parking_lot_yes']
X[columns].sum()

In [ ]:
X[columns].sum().sum()

The sum is 130, which means each of these 130 restaurants actually only has one parking option (there were restaurants without ratings that have multiple parking options).

In [ ]:
user_info.transport.value_counts()

In [ ]:
# mean rating as a function of parking option
plot_mean_rating(X[columns], rotate=True)

In [ ]:
# mean rating as a function of parking option, split into car owners and
# non car owners
y = plot_mean_rating_split(X[columns],user_info.transport,rotate=True)

print('no car:')
print(y['no car'])
print('\ncar owner:')
print(y['car owner'])

**Observations: ** (1) Surprisingly, public parking is the least popular one, and it's worse than no parking. (2) Valet parking is the most popular one, although there are only three restaurants with this option. (3) Food rating changes the least among all three types of ratings, which makes sense since it shouldn't be affected by parking options at all. (4) The trend of ratings from car owners is actually not very different from non car owners.
<br><br>
Is it possible that parking options don't matter? Let's see if the trend we see above can be explained by **price**, as we found previously that restaurants with medium or high price generally get higher ratings.


In [ ]:
X.price[X['parking_lot_valet parking']==1].value_counts()

In [ ]:
X.price[X['parking_lot_public']==1].value_counts()

In [ ]:
X.price[X['parking_lot_none']==1].value_counts()

**Observations: ** Apparently the restaurants with valet parking are the expensive/high-class ones, which is why they are highly rated. The compositions of the restaurants with public parking and no parking are similar; about half of either group are low-price restaurants. So the different ratings between these two groups given by car owners might be real. Maybe it's because public parking is expensive? 

### Smoking Area

In [ ]:
X.smoking_area.value_counts()

In [ ]:
user_info.smoker.value_counts()

In [ ]:
# mean rating as a function of smoking area
plot_mean_rating(X[['smoking_area']])

In [ ]:
# mean rating as a function of smoking area, split into smokers and
# non smokers
y = plot_mean_rating_split(X[['smoking_area']],user_info.smoker)

print('false:')
print(y['false'])
print('\ntrue:')
print(y['true'])

**Observations: ** (1) Non smokers do not care much about smoking area. (2) For smokers, **only at bar** is very popular, while **not permitted** leads to very low ratings. (3) Surprisingly, **none** and **not permitted** have very different mean ratings. Originally I thought these two mean the same thing. It's probably because smokers can still find a place to smoke at those restaurants without a smoking area but are completely not allowed to do so at those **not permitted** ones.

### Alcohol

In [ ]:
X.alcohol.value_counts()

In [ ]:
user_info.drink_level.value_counts()

In [ ]:
# mean rating as a function of alcohol
plot_mean_rating(X[['alcohol']])

In [ ]:
# mean rating as a function of smoking alcohol, split into abstemious, 
# casual and social drinkers
y = plot_mean_rating_split(X[['alcohol']],user_info.drink_level)

print('abstemious:')
print(y['abstemious'])
print('\ncasual drinker:')
print(y['casual drinker'])
print('\nsocial drinker:')
print(y['social drinker'])

**Observations: ** Abstemious drinkers prefer **Wine-Beer**; casual drinkers prefer **Full_Bar**; but it's not so clear for social drinkers.
<br><br>
Let's check again if the trend we see above can be explained by **price**.

In [ ]:
X.price[X.alcohol == 'No_Alcohol_Served'].value_counts()

In [ ]:
X.price[X.alcohol == 'Wine-Beer'].value_counts()

In [ ]:
X.price[X.alcohol == 'Full_Bar'].value_counts()

**Observations: ** On average, the order of price is **Full_Bar** > **Wine-Beer** > **No_Alcohol_Served**, which does not explain the ratings drinkers of abstemious and social drinkers.

### Other Services

In [ ]:
X.other_services.value_counts()

In [ ]:
# mean rating as a function of other_services
plot_mean_rating(X[['other_services']])

**Observations: ** The order of rating is **Variety** > **Internet** > **None**, consistent with the amount of other service provided.
<br><br>
Again, is it just caused by **price**?

In [ ]:
X.price[X.other_services == 'variety'].value_counts()

In [ ]:
X.price[X.other_services == 'Internet'].value_counts()

In [ ]:
X.price[X.other_services == 'none'].value_counts()

**Observations: ** Restaurants with **Internet** actually have a higher mean price than those with **Variety**. So the ratings are indeed affected by the amount of other service provided.

### Dress Code

In [ ]:
X.dress_code.value_counts()

In [ ]:
# mean rating as a function of dress code
plot_mean_rating(X[['dress_code']])

In [ ]:
# mean rating as a function of smoking alcohol, split into different 
# groups of consumers with different dress preferences
y = plot_mean_rating_split(X[['dress_code']],user_info.dress_preference)

print('no preference:')
print(y['no preference'])
print('\ninformal:')
print(y['informal'])
print('\nformal:')
print(y['formal'])

**Observations: ** Restaurants that require the customers to dress formally have the highest ratings. This is not a surprise because they should be expensive/high-class restaurants. Again, we can check the corresponding price in the following.

In [ ]:
X.price[X.dress_code == 'formal'].value_counts()

In [ ]:
X.price[X.dress_code == 'informal'].value_counts()

In [ ]:
X.price[X.dress_code == 'casual'].value_counts()

### Accessibility

In [ ]:
X.accessibility.value_counts()

In [ ]:
# mean rating as a function of dress accessibility
plot_mean_rating(X[['accessibility']])

**Observations: ** If accessibility could affect the rating, we would expect **completely > partially > no_accessibility**. However, this is not we see here. In the following, we can see that the trend is simply due to price again.

In [ ]:
X.price[X.accessibility == 'partially'].value_counts()

In [ ]:
X.price[X.accessibility == 'no_accessibility'].value_counts()

In [ ]:
X.price[X.accessibility == 'completely'].value_counts()

### Area

In [ ]:
X.area.value_counts()

In [ ]:
# mean rating as a function of area
plot_mean_rating(X[['area']])

**Observations: ** Food rating does not depend on **area**, but from overall and service ratings, we can tell that consumers still prefer closed area over open area.

### Cuisine

In [ ]:
# the number of restaurants for each cuisine type 
X.iloc[:,:23].sum()

In [ ]:
# Besides Mexican, Cafeteria and Fast_Food, group the other types into 
# the following 5 types

X['Rcuisine_Bar_Pub'] = np.zeros(X.shape[0])
index = ((X.Rcuisine_Bar == 1) | (X.Rcuisine_Bar_Pub_Brewery == 1))
X.loc[index,'Rcuisine_Bar_Pub'] = 1

X['Rcuisine_Asian'] = np.zeros(X.shape[0])
index = ((X.Rcuisine_Chinese == 1) | (X.Rcuisine_Japanese == 1) | (X.Rcuisine_Vietnamese == 1))
X.loc[index,'Rcuisine_Asian'] = 1

X['Rcuisine_Western'] = np.zeros(X.shape[0])
index = ((X.Rcuisine_Armenian == 1) | (X.Rcuisine_Italian == 1) | (X.Rcuisine_Mediterranean == 1) 
         | (X.Rcuisine_Pizzeria == 1) | (X.Rcuisine_Seafood == 1))
X.loc[index,'Rcuisine_Western'] = 1

X['Rcuisine_American_Burgers'] = np.zeros(X.shape[0])
index = ((X.Rcuisine_American == 1) | (X.Rcuisine_Burgers == 1))
X.loc[index,'Rcuisine_American_Burgers'] = 1

X['Rcuisine_Others'] = np.zeros(X.shape[0])
index = (((X.Rcuisine_Bakery == 1) | (X["Rcuisine_Breakfast-Brunch"] == 1) | (X["Rcuisine_Cafe-Coffee_Shop"] == 1) 
        | (X.Rcuisine_Contemporary == 1) | (X.Rcuisine_Family == 1) | (X.Rcuisine_Game == 1) 
        | (X.Rcuisine_International == 1) | (X.Rcuisine_Regional == 1)))
X.loc[index,'Rcuisine_Others'] = 1

In [ ]:
print("Number of restaurants for each type")
print("Mexican: {}".format(int(X.Rcuisine_Mexican.sum())))
print("American/Burgers: {}".format(int(X.Rcuisine_American_Burgers.sum())))
print("Asian: {}".format(int(X.Rcuisine_Asian.sum()))) 
print("Bar/Pub: {}".format(int(X.Rcuisine_Bar_Pub.sum()))) 
print("Cafeteria: {}".format(int(X.Rcuisine_Cafeteria.sum()))) 
print("Fast Food: {}".format(int(X.Rcuisine_Fast_Food.sum()))) 
print("Others: {}".format(int(X.Rcuisine_Others.sum())))
print("Western: {}".format(int(X.Rcuisine_Western.sum())))

In [ ]:
columns = ['Rcuisine_Mexican','Rcuisine_American_Burgers', 'Rcuisine_Asian','Rcuisine_Bar_Pub', 'Rcuisine_Cafeteria',
           'Rcuisine_Fast_Food','Rcuisine_Others','Rcuisine_Western']
plot_mean_rating(X[columns],rotate=True)

Let's also check the correlation coefficients between these cuisine types and the ratings

In [ ]:
X_cuisine = X[['Rcuisine_Mexican','Rcuisine_American_Burgers','Rcuisine_Asian','Rcuisine_Bar_Pub','Rcuisine_Cafeteria',
           'Rcuisine_Fast_Food','Rcuisine_Western','Rcuisine_Others']]

X_cuisine['Y_food'] = Y_food_mean
X_cuisine['Y_overall'] = Y_overall_mean
X_cuisine['Y_service'] = Y_service_mean

In [ ]:
print(X_cuisine.corr()['Y_overall'][:-3])
print('\n')
print(X_cuisine.corr()['Y_food'][:-3])
print('\n')
print(X_cuisine.corr()['Y_service'][:-3])
print('\n')

**Observations: ** Although the correlations are slightly different between different ratings, **Rcuisine_Others** generally gets higher ratings and **Rcuisine_Fast_Food** generally gets lower ratings. Asian restaurants perform really well at food rating. Interestingly, Mexican restaurants have above than average food rating but just roughly average overall and service ratings.  

## Conclusion

So what features affect the ratings? 
- **Price** is definitely one. Generally medium and high-price restaurants have better rating. However, it may be better to transform the original categories (low, medium, high) using dummy variables (one-hot encoding) instead of directly mapping them to numbers (such as 1,2,3). This is because we saw that people with different budget actually react differently on the price change.
<br><br>
- **Parking options** can affect the ratings too, but not the food rating. Also, it only matters to car owners. It's probably only necessary to distinguish between public parking and the other options (including no parking) because we saw that restaurants with public parking have significantly lower overall and service ratings.
<br><br>
- **Smoking Area** is only important for smokers. Especially, restaurants that do not allow smoking are low rated, and restaurants with their smoking area located at bar are highly rated.
<br><br>
- **Alcohol** is also relevant but different types of drinkers react differently on the change of alcohol service (No_Alcohol_Served, Wine-Beer, Full_Bar). Similar to **Price**, it may be better to transform the original three categories using dummy variables.
<br><br>
- **Other Services** has influence on the ratings. We saw that restaurants with a variety of other service perform better than those with only Internet. And those with Internet are better than those without other services. 
<br><br>
- The effect of **Dress Code** on the ratings is probably due to **Price**. However, there are only two restaurants that require the customers to dress formally and their ratings are close to 2. I would keep a feature (0 or 1) to show whether a restaurant has formal dress code because when this condition is true, the ratings are essentially guaranteed to be very good.
<br><br>
- **Accessibility** does not affect the ratings. For **Area**, people prefer closed space over open space. For **Cuisine**, what restaurant/food types are relevant depend on which rating we are trying to predict. But in general, some types have significant influence. For example, fast food restaurants generally have low ratings. 